In [1]:
from crewai import Crew, Task, Agent
import os
import warnings

warnings.filterwarnings("ignore")
os.environ["OPENAI_API_KEY"] = os.environ["OPENAI_API_KEY_PERSONAL"]
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

## Creating Agents

In [2]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory="As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth.",
    allow_delegation=False,
    verbose=True
)

In [3]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory="Within the vibrant ecosystem of CrewAI's sales department, "
              "you stand out as the bridge between potential clients "
              "and the solutions they need."
              "By creating engaging, personalized messages, "
              "you not only inform leads about our offerings "
              "but also make them feel seen and heard."
              "Your role is pivotal in converting interest "
              "into action, guiding leads through the journey "
              "from curiosity to commitment.",
    allow_delegation=False,
    verbose=True
)

## Creating Tools

In [4]:
from crewai_tools import DirectoryReadTool, FileReadTool, SerperDevTool

/opt/homebrew/lib/python3.11/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/opt/homebrew/lib/python3.11/site-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("website_url")
/opt/homebrew/lib/python3.11/site-packages/crewai_tools/tools/selenium_scraping_tool/selenium_scraping_tool.py:26: PydanticDeprecatedSince20: Pydantic V1 style `@v

In [5]:
directory_read = DirectoryReadTool("./instructions/")
file_read = FileReadTool()
search_tool = SerperDevTool()

## Custom Tool

In [7]:
from crewai.tools import BaseTool

In [8]:
class SentimentAnalysisTool(BaseTool):
    name: str = "Sentiment Analysis Tool"
    description: str = "Analyzes the sentiment of text to ensure positive and engaging communication."

    def _run(self, text: str):
        return "positive"

In [9]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Task

In [10]:
lead_profiling_task = Task(
    description="Conduct an in-depth analysis of {lead_name}, "
                "a company in the {industry} sector "
                "that recently showed interest in our solutions. "
                "Utilize all available data sources "
                "to compile a detailed profile, "
                "focusing on key decision-makers, recent business "
                "developments, and potential needs "
                "that align with our offerings. "
                "This task is crucial for tailoring "
                "our engagement strategy effectively.\n"
                "Don't make assumptions and "
                "only use information you absolutely sure about.",
    expected_output="A comprehensive report on {lead_name}, "
                    "including company background, "
                    "key personnel, recent milestones, and identified needs. "
                    "Highlight potential areas where "
                    "our solutions can provide value, "
                    "and suggest personalized engagement strategies.",
    tools=[directory_read, file_read, search_tool],
    agent=sales_rep_agent
)

In [11]:
personalised_outreach_task = Task(
    description="Using the insights gathered from "
                "the lead profiling report on {lead_name}, "
                "craft a personalized outreach campaign "
                "aimed at {key_decision_maker}, "
                "the {position} of {lead_name}. "
                "The campaign should address their recent {milestone} "
                "and how our solutions can support their goals. "
                "Your communication must resonate "
                "with {lead_name}'s company culture and values, "
                "demonstrating a deep understanding of "
                "their business and needs.\n"
                "Don't make assumptions and only "
                "use information you absolutely sure about.",
    expected_output="A series of personalized email drafts "
                    "tailored to {lead_name}, "
                    "specifically targeting {key_decision_maker}."
                    "Each draft should include "
                    "a compelling narrative that connects our solutions "
                    "with their recent achievements and future goals. "
                    "Ensure the tone is engaging, professional, "
                    "and aligned with {lead_name}'s corporate identity.",
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent
)

## Creating the Crew

In [12]:
crew = Crew(
    agents = [sales_rep_agent, lead_sales_rep_agent],
    tasks = [lead_profiling_task, personalised_outreach_task],
    verbose = True,
    memory = True
)

In [13]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

In [14]:
result = crew.kickoff(inputs = inputs)

# Agent: Sales Representative
## Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


# Agent: Sales Representative
## Thought: I need to gather specific information about DeepLearningAI, including its background, key personnel, recent developments, and potential needs. I will start by searching the internet for updated information on DeepLearningAI.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"DeepLearningAI company profile key personnel recent developments needs\"}"
## Tool Output: 
{'searchParameters': {'q': 'DeepLearningAI 

In [16]:
from IPython.display import Markdown

In [18]:
Markdown(result.raw)

**Email Template 1: Collaboration Proposal**
```
Subject: Potential Collaboration in AI Education

Dear Andrew,

I hope this message finds you well! I admire DeepLearningAI's recent efforts in launching the “ChatGPT Prompt Engineering for Developers” course, which is a timely and valuable addition to the AI education landscape.

At [Your Company], we specialize in creating innovative analytics and management solutions tailored for educational platforms. Our offerings could significantly empower your educators by facilitating data-driven insights into learner engagement and course performance.

I would be thrilled to explore potential collaboration avenues to help advance your mission of accessible AI education.

Best regards,  
[Your Name]  
[Your Company]  
```

**Email Template 2: Integrative Technology Offer**
```
Subject: Enhancing DeepLearningAI’s Learning Experiences

Hi Andrew,

Congratulations on the impactful work DeepLearningAI is doing to bridge knowledge gaps in AI. I recently came across your new courses and wanted to reach out regarding an opportunity.

At [Your Company], we focus on developing tools that enhance online learning experiences through AI-driven personalization and analytics. We believe that our solutions can seamlessly integrate with your existing framework to provide real-time feedback to educators and learners alike.

Let's discuss how we can work together to enhance the learning experience for your community.

Looking forward to connecting!

Best,  
[Your Name]  
[Your Company]  
```

**Email Template 3: Exploring Synergies**
```
Subject: Exploring Synergies in AI Education

Dear Andrew,

I’m impressed by DeepLearningAI's dedication to transforming AI education. Your recent successes, particularly the collaboration with leading educational institutions, highlight your innovative approach.

I believe that [Your Company] can support your mission through our suite of educational technologies that enhance learning engagement and outcomes. I would love to chat about how we can potentially collaborate to build on DeepLearningAI’s excellent foundation.

Thank you for considering this opportunity. I look forward to your response!

Warm regards,  
[Your Name]  
[Your Company]  
```
```